# Etapa 6:  Predicción mediante aprendizaje automático

In [7]:
"""
1. Inicializar modelo
nombre_variable = Modelo ()

2. Búsqueda mejores HP
variable2 = GridSearch (nombre_variable, parameters)

3. Ajustar modelo
variable2.fit (X_train, Y_train)

4. Predecir
Y_pred = variable2.predict(X_test)

5. Comparar Y_test vs Y_pred
plot_confusion_matrix(Y_test, Y_pred)


"""

'\n1. Inicializar modelo\nnombre_variable = Modelo ()\n\n2. Búsqueda mejores HP\nvariable2 = GridSearch (nombre_variable, parameters)\n\n3. Ajustar modelo\nvariable2.fit (X_train, Y_train)\n\n4. Predecir\nY_pred = variable2.predict(X_test)\n\n5. Comparar Y_test vs Y_pred\nplot_confusion_matrix(Y_test, Y_pred)\n\n\n'

Space X anuncia en su página web lanzamientos de cohetes Falcon 9 con un coste de 62 millones de dólares; otros proveedores cuestan más de 165 millones de dólares cada uno, gran parte del ahorro se debe a que Space X puede reutilizar la primera etapa. Por lo tanto, si podemos determinar si la primera etapa aterrizará, podemos determinar el coste de un lanzamiento. Esta información puede utilizarse si una empresa alternativa quiere pujar contra Space X por el lanzamiento de un cohete.   En este laboratorio, crearás una línea de aprendizaje automático para predecir si la primera etapa aterrizará dados los datos de los laboratorios anteriores.

## Objetivos

Realizar un análisis exploratorio de los datos y determinar las etiquetas de formación

* Crear una columna para la clase
* Estandarizar los datos
* Dividir en datos de entrenamiento y datos de prueba

\-Encontrar el mejor hiperparámetro para SVM, árboles de clasificación y regresión logística

* Encontrar el método que mejor se comporta utilizando los datos de prueba


***


## Importar librerias y Definir Funciones Auxiliares


In [8]:
#IMPORTAR LIBRERÍAS
# (pd) PANDAS = PARA MOSTRAR TABLAS (DATAFRAME)
# (np) NUMPY = PARA USAR FUNCIONES NUMÉRICAS
# (plt) MATPLOTLIB = PARA USAR GRÁFICOS
# (sns) SEABORN = PARA GRÁFICOS MEJORADOS DE MATPLOTLIB
# SKLEARN = MACHINE LEARNING PARA ALGORITMOS Y APRENDIZAJE AUTOMÁTICO
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

Esta función sirve para visualizar la matriz de confusión.


In [9]:
# FUNCIÓN PARA VISUALIZAR MATRIZ DE CONFUSIÓN
def plot_confusion_matrix(y,y_predict):
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax);
    ax.set_xlabel('Valores predecidos')
    ax.set_ylabel('Valores reales')
    ax.set_title('Confusion Matrix');
    ax.xaxis.set_ticklabels(['no aterrizó', 'aterrizó']); ax.yaxis.set_ticklabels(['no aterrizó', 'aterrizó'])

## Cargar el dataframe


Cargar los datos


In [10]:
#CARGA DATAFRAME (pd) PANDAS DATASET.PART2
#VER 5 PRIMERAS FILAS CON HEAD
data = pd.read_csv("dataset_part_2.csv")
data.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,...,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,...,False,NaN,1.0,SpaceX,True,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,...,False,NaN,1.0,NASA(COTS),True,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,CRS-2,ISS,CCSFS SLC 40,None None,1,False,...,False,NaN,1.0,NASA (CRS),True,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,...,False,NaN,1.0,MDA,True,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,SES-8,GTO,CCSFS SLC 40,None None,1,False,...,False,NaN,1.0,SES,True,0,B1004,-80.577366,28.561857,0


Al conjunto de datos anterior hemos aplicado una transformación one-hot para poder realizar las tareas de machine learning correctamente

In [11]:
X = pd.read_csv('dataset_one-hot.csv')



## Tarea  1


Crea un array NumPy a partir de la columna `Class` en `data`, aplicando el método `to_numpy()` y luego
asignarlo a la variable `Y`, asegúrese de que la salida es un array de Numpy (sólo un paréntesis df\['nombre de la columna'\]).

In [12]:
#Inserta tu código aquí
Y = data['Class'].to_numpy()


## Tarea  2


Normalice los datos en `X` usando al función `fit_transform` y luego reasígnelos a la variable `X` utilizando la transformación proporcionada a continuación.


In [13]:
scaler = preprocessing.StandardScaler()

In [14]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

Dividimos los datos en datos de entrenamiento y de prueba utilizando la función `train_test_split`.   Los datos de entrenamiento se dividen en datos de validación, un segundo conjunto utilizado para los datos de entrenamiento; luego se entrenan los modelos y se seleccionan los hiperparámetros utilizando la función `GridSearchCV`.


## Tarea  3


Utilice la función `train_test_split` para dividir los datos X e Y en datos de entrenamiento y de prueba como hicimos en la práctica de regresión. Establezca el parámetro test_size en 0,2 y random_state en 2. Los datos de entrenamiento y los datos de prueba deben asignarse a las siguientes etiquetas.


`X_train, X_test, Y_train, Y_test`


In [15]:
#Inserta tu código aquí
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

## Tarea  4


* Cree un objeto de regresión logística.
* Luego crea un objeto GridSearchCV `logreg_cv` con el objeto de regresión lógistica, los parametros descritos, `scoring='accuracy'` y `cv = 10`.  
* Ajusta el objeto GridSearchCV para encontrar los mejores parámetros del diccionario `parameters`.


In [16]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [17]:
# Crear objeto de regresión logística
modelo = LogisticRegression()

# Definir los parámetros para GridSearchCV parameters arriba

# Crear objeto GridSearchCV
logreg_cv = GridSearchCV(modelo, parameters, scoring='accuracy', cv=10)

# Ajustar GridSearchCV para encontrar los mejores parámetros
logreg_cv.fit(X_train, Y_train)



GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1], 'penalty': ['l2'],
                         'solver': ['lbfgs']},
             scoring='accuracy')

Mostramos los mejores parámetros utilizando el atributo de datos `best_params_` y la precisión en los datos de validación utilizando el atributo de datos `best_score_`.


In [18]:
print("Mejores parámetros ", logreg_cv.best_params_)
print("Precisión de la regresión logística:", logreg_cv.best_score_)

Mejores parámetros  {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Precisión de la regresión logística: 0.8214285714285714


## Tarea  5


Calcule la precisión en los datos de prueba utilizando el método `score`sobre `X_test` e `Y_test`:


In [19]:
#Inserta tu código aquí
accuracy = logreg_cv.score(X_test, Y_test)
print("Precisión en los datos de prueba:", accuracy)

Precisión en los datos de prueba: 0.8333333333333334


Predice `X_test` y guardalo en `y_pred`

In [20]:
#Inserta tu código aquí
logreg.fit(X_train, Y_train)
y_pred = logreg.predict(X_test)

NameError: ignored

Compara `y_pred` con `Y_test` con una matriz de confusión:

In [ ]:
#Inserta tu código aquí
# Calcula la matriz de confusión
cm = plot_confusion_matrix(Y_test, y_pred)

# Imprime la matriz de confusión
print("Matriz de confusión:")
print(cm)

Al examinar la matriz de confusión, vemos que la regresión logística puede distinguir entre las diferentes clases.  Vemos que el principal problema son los falsos positivos.


## Tarea  6


* Crea un objeto [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html) y guardalo en svm
* Luego crea un objeto `GridSearchCV` `svm_cv` con el objeto de svm, los parametros descritos, `scoring='accuracy'` y  `cv=10`.  
* Ajusta el objeto para encontrar los mejores parámetros del diccionario `parameters`.


In [ ]:
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}

In [ ]:
#Inserta tu código aquí
svm = SVC()

In [ ]:
#Inserta tu código aquí
svm_cv = GridSearchCV(svm, parameters, scoring='accuracy', cv=10)

In [ ]:
# Ajusta el objeto GridSearchCV para encontrar los mejores parámetros
svm_cv.fit(X_train, Y_train)

# Obtiene los mejores parámetros encontrados
print("Mejores parámetros: ", svm_cv.best_params_)
print("Accuracy :", svm_cv.best_score_)

## Tarea  7


Calcule la precisión en los datos de prueba utilizando el método `score`sobre `X_test` e `Y_test`:


In [ ]:
# Calcula la precisión en los datos de prueba
accuracy = svm_cv.score(X_test, Y_test)

# Imprime la precisión
print("Precisión en los datos de prueba:", accuracy)


Podemos representar la matriz de confusión:

In [ ]:
yhat=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

## Tarea  8


* Crea un objeto clasificador de árbol de decisión
* Y luego crea un objeto GridSearchCV tree_cv con el objeto clasificador de árbol de decisión, los parametros descritos, `scoring='accuracy'` y  cv = 10.
* Ajuste el objeto para encontrar los mejores parámetros a partir de los parámetros del diccionario.

In [ ]:
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

In [ ]:
# Crea el objeto clasificador de árbol de decisión
tree = DecisionTreeClassifier()

# Crea el objeto GridSearchCV
tree_cv = GridSearchCV(tree, parameters, scoring='accuracy', cv=10)

# Ajusta el objeto GridSearchCV para encontrar los mejores parámetros
tree_cv.fit(X_train, Y_train)

# Obtiene los mejores parámetros encontrados
best_params = tree_cv.best_params_
print("Mejores parámetros:", best_params)

In [ ]:
print("Mejores parámetros:", tree_cv.best_params_)
print("Accuracy for decision tree classifier:", tree_cv.best_score_)

## Tarea  9


Calcula la precisión de tree_cv en los datos de prueba utilizando el método `score`:


In [ ]:
# Calcula la precisión en los datos de prueba
accuracy = tree_cv.score(X_test, Y_test)

# Imprime la precisión
print("Precisión en los datos de prueba:", accuracy)


Podemos representar la matriz de confusión


In [ ]:
yhat = tree_cv.predict(X_test)
plot_confusion_matrix(Y_test, yhat)

## Tarea  10


* Crear un objeto k vecinos más cercanos
* Y luego crear un objeto `GridSearchCV` `knn_cv` con el objeto k vecinos más cercanos, los parametros descritos, `scoring='accuracy'` y  cv = 10.
* Ajuste el objeto para encontrar los mejores parámetros del diccionario `parameters`.


In [ ]:
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

In [ ]:
# Crea el objeto de clasificador de k vecinos más cercanos
knn = KNeighborsClassifier()


In [ ]:
# Crea el objeto GridSearchCV
knn_cv = GridSearchCV(knn, parameters, scoring='accuracy', cv=10)



In [ ]:
# Ajusta el objeto GridSearchCV para encontrar los mejores parámetros
knn_cv.fit(X_train, Y_train)

# Obtiene los mejores parámetros encontrados
print("Mejores parámetros: ",knn_cv.best_params_)
print("Accuracy for k nearest neighbors:",knn_cv.best_score_)

## Tarea  11


Calcula la precisión de knn_cv en los datos de prueba utilizando el método `score`:


In [ ]:
# Calcula la precisión en los datos de prueba
accuracy = knn_cv.score(X_test, Y_test)

# Imprime la precisión
print("Precisión en los datos de prueba:", accuracy)


Podemos representar la matriz de confusión


In [ ]:
yhat = knn_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

## Tarea  12


Encuentra el método que mejor funciona. Para ello crea un dataframe con los los resultados y los algoritmos. Muestra los diferentes resultado en un gráfico de barras.

In [ ]:
algorithms = {'KNN':knn_cv.best_score_,'Decision Tree':tree_cv.best_score_,'Logistic Regression':logreg_cv.best_score_,'SVM':svm_cv.best_score_}


In [ ]:
#Inserta aquí tu código new
algo_df = pd.DataFrame.from_dict(algorithms, orient='index', columns=['Accuracy'])

#Resetea el indice de el df
algo_df.reset_index(inplace=True)

#Renombre las columnas con: {'index': 'Algorithm'}
algo_df.rename(columns={'index': 'Algorithm'}, inplace=True)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.bar(algo_df, x='Algorithm', y='Accuracy', hover_data=['Algorithm', 'Accuracy'], color='Accuracy', color_continuous_scale='rdylbu')
fig.update_layout(title='Algorithm vs. Accuracy', xaxis_title='Algorithm', yaxis_title='Accuracy' )
fig.show()